In [93]:
import glob
import re
import mne
import itertools
import numpy as np
import matplotlib.pyplot as plt

In [115]:
N_CHANS = 62
FS = 5000

In [ ]:
epochs_path = glob.glob('../data/bids/derivatives/preprocessing/*/*res-hi*')
pattern = re.compile('sub-(\d{1,2})_')
epoch_sub_order = list(map(pattern.findall, epochs_path))
epoch_sub_order = list(itertools.chain(*epoch_sub_order))

captrak_path = glob.glob('../data/captrak/*.bvct')
# pattern = re.compile('subj-(\d{1,2})')
# captrak_sub_order = list(map(pattern.findall, captrak_path))
# captrak_sub_order = list(itertools.chain(*captrak_sub_order))

# n_subs = len(captrak_path)
# poss = np.empty([n_subs, N_CHANS, 3])
poss = []
# for each captrak file
for file in captrak_path:
    # get subject
    pattern = re.compile('subj-(\d{1,2})')
    sub = pattern.findall(fpath)[0]
    
    # get corresponding captrak file
    dig = mne.channels.read_dig_captrak(fname = file)
    
    # load their epochs object
    ind = epoch_sub_order.index('43')
    epoch_fpath = epochs_path[ind]
    epochs = mne.read_epochs(epoch_fpath, preload = False)

    # get coordinates
    ch_names = epochs.ch_names
    if len(ch_names) != N_CHANS:
        print(f"Incorrect number of channels, skipping subjet {sub}")
        continue
    pos = np.stack([dig.get_positions()['ch_pos'][ch] for ch in ch_names])
    poss = [poss, pos]
    
# Plot mean coordinates
# rng = np.random.RandomState(0)
# data = rng.normal(size=(N_CHANS, 1)) * 1e-6
# fake_evoked = mne.EvokedArray(data, epoch.info)
# fake_evoked.set_montage(dig)
# fake_evoked.plot_sensors()
    

Reading /project2/hcn1/pitch_tracking/scripts/../data/bids/derivatives/preprocessing/sub-43/sub-43_task-pitch_run-1_res-hi_desc-clean_epo.fif-1.gz ...


/tmp/ipykernel_10485/2372708796.py:26: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-43/sub-43_task-pitch_run-1_res-hi_desc-clean_epo.fif-1.gz) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epoch_fpath, preload = False)


    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available


In [114]:
epoch_sub_order.index('43')

24

In [4]:
path = '../data/bids/derivatives/preprocessing/sub-9/*'
for fpath in glob.glob(path):
    print(fpath)

# epochs = mne.read_epochs(FPATH, preload = True)
# events = epochs.events
# n_epochs = len(events)

# for file in epochs_path:
#     captrak_path = 
# # print(sorted(epochs_path))
# # print(sorted(captrak_path))
# epochs_path


# for file in epochs

../data/bids/derivatives/preprocessing/sub-9/sub-9_task-pitch_run-1_res-lo_desc-clean_epo.fif.gz
../data/bids/derivatives/preprocessing/sub-9/sub-9_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz


In [7]:
fpath = '../data/bids/derivatives/preprocessing/sub-9/sub-9_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz'
epochs = mne.read_epochs(fpath, preload = False)

Reading /project2/hcn1/pitch_tracking/scripts/../data/bids/derivatives/preprocessing/sub-9/sub-9_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Not setting metadata
1922 matching events found
No baseline correction applied
0 projection items activated


In [100]:
captrak_fpath = '../data/captrak/subj-9.bvct'
dig = mne.channels.read_dig_captrak(fname = captrak_fpath)
ch_names = epochs.ch_names
pos = np.stack([dig.get_positions()['ch_pos'][ch] for ch in ch_names])

In [28]:
# T7_index = ch_names.index('T7')
# T8_index = ch_names.index('T8')
# radius = np.abs(pos[[T7_index, T8_index], 0]).mean()

In [105]:
n_channels = 62
n_samples = 1
rng = np.random.RandomState(0)
data = rng.normal(size=(n_channels, 1)) * 1e-6
# info = mne.create_info(62, 5000, ch_types = 'eeg')
fake_evoked = mne.EvokedArray(data, epoch.info)
fake_evoked.set_montage(dig)
fake_evoked.plot_sensors()

ValueError: DigMontage is only a subset of info. There are 62 channel positions not present in the DigMontage. The required channels are:

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61'].

Consider using inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.

In [102]:
fake_evoked.info

<Info | 8 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, ...
 chs: 62 misc
 custom_ref_applied: False
 dig: 3 items (3 Cardinal)
 highpass: 0.0 Hz
 lowpass: 2500.0 Hz
 meas_date: unspecified
 nchan: 62
 projs: []
 sfreq: 5000.0 Hz
>